In [ ]:
import pandas as pd
import numpy as np
import datetime
import json

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

train_ = pickle.load(open('sales_data/train_7days.pkl', 'rb'))
val_ = pickle.load(open('sales_data/val_7days.pkl', 'rb'))
test_ = pickle.load(open('sales_data/test_7days.pkl', 'rb'))

train_idx, val_idx, test_idx = list(train_.keys()), list(val_.keys()), list(test_.keys())

abs_data = pd.concat([
    pd.read_csv('train_bert_data/train_7days.csv'),
    pd.read_csv('train_bert_data/val_7days.csv'),
    pd.read_csv('train_bert_data/test_7days.csv')
])
print(abs_data.shape)

In [ ]:
min_date, max_date = datetime.datetime(2013, 1, 1), datetime.datetime(2017, 12, 31)
days_len = 7
items = sorted(train['item'].drop_duplicates())

train_dict, val_dict, test_dict = dict(), dict(), dict()
start_date = min_date

In [ ]:
train.loc[train['item'] == item].loc[train['date'] >= start_str].loc[train['date'] < end_str].sort_values(by=['date']).pivot(index='date', columns='store', values='sales').sum(1).values

In [ ]:
list(filter(lambda x: f':item{item}-' in x, abs_data.loc[abs_data['date'] == dd.strftime('%Y-%m-%d'), 'seq_cnv']))

In [ ]:
%%time
import numpy as np
import datetime
import json

train_ = pickle.load(open('sales_data/train_7days.pkl', 'rb'))
val_ = pickle.load(open('sales_data/val_7days.pkl', 'rb'))
test_ = pickle.load(open('sales_data/test_7days.pkl', 'rb'))

train_idx, val_idx, test_idx = list(train_.keys()), list(val_.keys()), list(test_.keys())

abs_data = pd.concat([
    pd.read_csv('train_bert_data/train_7days.csv'),
    pd.read_csv('train_bert_data/val_7days.csv'),
    pd.read_csv('train_bert_data/test_7days.csv')
])
print(abs_data.shape)

min_date, max_date = datetime.datetime(2013, 1, 1), datetime.datetime(2017, 12, 31)
days_len = 7
items = sorted(train['item'].drop_duplicates())

train_dict, val_dict, test_dict = dict(), dict(), dict()
start_date = min_date
while True:
    end_date = start_date + datetime.timedelta(days=days_len)
    if end_date > max_date:
        break
    else:
        start_str, end_str = start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')
        for item in items:
            abs_parts = []
            dd = start_date
            while dd < end_date:
                abs_parts.append(list(filter(lambda x: f':item{item}-' in x, abs_data.loc[abs_data['date'] == dd.strftime('%Y-%m-%d'), 'seq_cnv'])))
                dd += datetime.timedelta(days=1)
            input_parts = train.loc[train['item'] == item].loc[train['date'] >= start_str].loc[train['date'] < end_str].sort_values(by=['date'])
            input_pivot = input_parts.pivot(index='date', columns='store', values='sales')
            output_parts = train.loc[train['item'] == item].loc[train['date'] == end_str].pivot(index='date', columns='store', values='sales')
            part = {
                'store_seq': abs_parts,
                'input': list(map(lambda x: list(map(lambda y: int(y), list(x))), input_pivot.values.T)),
                'output': list(map(lambda x: int(x), output_parts.values.squeeze())),
                'total': list(map(lambda x: int(x), input_pivot.sum(1).values))
            }
            if start_str in train_idx:
                if item not in train_dict.keys():
                    train_dict[item] = dict()
                train_dict[item][start_str] = part
            elif start_str in val_idx:
                if item not in val_dict.keys():
                    val_dict[item] = dict()
                val_dict[item][start_str] = part
            else:
                if item not in test_dict.keys():
                    test_dict[item] = dict()
                test_dict[item][start_str] = part
    print(start_str, len(train_dict), len(val_dict), len(test_dict))
    start_date += datetime.timedelta(days=1)

In [ ]:
filenames = list(map(lambda x: f'sales_data/{x}_7days.json', ['train', 'val', 'test']))
for fi in filenames:
    with open(fi, 'w') as f:
        if 'train' in fi:
            json.dumps(train_dict, f)
        elif 'val' in fi:
            json.dumps(val_dict, f)
        else:
            json.dumps(test_dict, f)

In [ ]:
val = json.loads(open('sales_data/val_7days.json', 'r'))

In [ ]:
import json 
train_dict, test_dict = dict(), dict()
for k in item_sales_dict.keys():
    if k < '2016-01-01':
        train_dict[k] = item_sales_dict[k]
    else:
        test_dict[k] = item_sales_dict[k]
len(item_sales_dict), len(train_dict), len(test_dict)

In [ ]:
val_dict = dict()
val_dates = np.random.choice(list(train_dict.keys()), round(len(train_dict) * 0.1), replace=False)
val_dates

In [ ]:
for d in val_dates:
    val_dict[d] = train_dict[d]
    del train_dict[d]
len(train_dict), len(val_dict)

In [ ]:
import pickle
train = pickle.load(open('sales_data/train_7days.pkl', 'rb'))
val = pickle.load(open('sales_data/val_7days.pkl', 'rb'))
test = pickle.load(open('sales_data/test_7days.pkl', 'rb'))
val.keys()

In [ ]:
abs_data = pd.concat([
    pd.read_csv('train_bert_data/train_7days.csv'),
    pd.read_csv('train_bert_data/val_7days.csv'),
    pd.read_csv('train_bert_data/test_7days.csv')
])
abs_data.shape

In [ ]:
for store in train['store'].drop_duplicates():
    print(f'train store: {store}, items: {", ".join(train.loc[train["store"] == store, "item"].drop_duplicates().astype(str))}')
    print(f'test store: {store}, items: {", ".join(test.loc[test["store"] == store, "item"].drop_duplicates().astype(str))}')

In [ ]:
import numpy as np
min_sales, max_sales = 0, 300
range_ = 10
sales_blocks = dict()
base = ord('a')
for j, i in enumerate(np.arange(min_sales, max_sales, range_)):
    low, high = i, i + range_ - 1
    sales_blocks[(low, high)] = chr(base + j)
sales_blocks[(max_sales, 1000000)] = chr(base + len(sales_blocks))
sales_blocks

In [ ]:
def convert_code(sales_blocks, x):
    for k, v in sales_blocks.items():
        if x >= k[0] and x <= k[1]:
            return sales_blocks[k]

train['sales_processed'] = list(map(lambda x: convert_code(sales_blocks, x), train['sales']))
train.head()


In [ ]:
train.to_csv('data/train_converted.csv', index=False)

In [ ]:
%%time

import datetime

min_date, max_date = datetime.datetime(2013, 1, 1), datetime.datetime(2017, 12, 31)
days_len = 7
train['sales'] = train['sales'].astype(str)

for store in train['store'].drop_duplicates():
    start_time = datetime.datetime.now()
    dates, inputs_converted, inputs_org = [], [], []
    start_date = min_date
    while True:
        end_date = start_date + datetime.timedelta(days=days_len)
        if end_date > max_date:
            break

        start_str, end_str = start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')
        input_parts = train.loc[train['store'] == store].loc[train['date'] >= start_str].loc[train['date'] < end_str].sort_values(by=['date', 'item'])
        output_parts = train.loc[train['store'] == store].loc[train['date'] == end_str]
        input_ = f'{store}:{start_str[5:]}'
        for item in input_parts['item'].drop_duplicates():
            inputC = f'date{start_str[5:]}:store{store}:item{item}-' + ''.join(input_parts.loc[input_parts['item'] == item, 'sales_processed'].values)
            inputO = f'date{start_str[5:]}:store{store}:item{item}-' + '>'.join(input_parts.loc[input_parts['item'] == item, 'sales'].values)
            dates.append(start_str)
            inputs_converted.append(inputC)
            inputs_org.append(inputO)
        start_date = start_date + datetime.timedelta(days=1)
    records = pd.DataFrame({'date': dates, 'seq_org': inputs_org, 'seq_cnv': inputs_converted})
    records.to_csv(f'data/store{store}_{days_len}days_converted.csv', index=False)
    print(f'dur: {datetime.datetime.now() - start_time}')

In [ ]:
n_store, std_date = 10, '2016-01-01'
train, test = [], []
for i in range(n_store):
    whole = pd.read_csv(f'data/store{i + 1}_{days_len}days_converted.csv')
    train.append(whole.loc[whole['date'] < std_date])
    test.append(whole.loc[whole['date'] >= std_date])
train = pd.concat(train).reset_index(drop=True)
test = pd.concat(test).reset_index(drop=True)
val_idx = np.random.choice(np.arange(len(train)), round(len(train) * 0.1), replace=False)
train_idx = set(np.arange(len(train))) - set(val_idx)
val = train.iloc[sorted(val_idx)]
train = train.iloc[sorted(train_idx)]
train.to_csv(f'train_bert_data/train_{days_len}days.csv', index=False)
val.to_csv(f'train_bert_data/val_{days_len}days.csv', index=False)
test.to_csv(f'train_bert_data/test_{days_len}days.csv', index=False)

In [ ]:
import os
from transformers import BertForMaskedLM, AutoTokenizer, pipeline
from skt.vault_utils import get_secrets

proxy = get_secrets("proxy")["proxy"]
os.environ['HTTP_PROXY'] = proxy
os.environ['HTTPS_PROXY'] = proxy

bert = BertForMaskedLM.from_pretrained('distilbert-base-uncased').cuda()
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
fill_mask = pipeline(
    "fill-mask",
    model=bert,
    tokenizer=tokenizer
)

In [ ]:
bert.device

In [ ]:
bert.cpu()
bert.device

In [ ]:
bert.device

In [ ]:
import torch
inputs = tokenizer("Hello, my [MASK] is cute", return_tensors="pt")
print(inputs.input_ids)
with torch.no_grad(): 
    outputs = bert(**inputs)
outputs.prediction_logits

In [ ]:
text = torch("The body was recovered and sent to Cox's Bazar [MASK] morgue for autopsy")
text.cuda()
#fill_mask("The body was recovered and sent to Cox's Bazar [MASK] morgue for autopsy")

In [ ]:
fill_mask("Hello, my [MASK] is cute")[0]['sequence'][6:-6]

In [ ]:
val.head()

In [ ]:
input_ids = tokenizer(list(val['seq_org'])).input_ids

In [ ]:
len(input_ids[0])

In [ ]:
import os
from skt.vault_utils import get_secrets
from transformers import RobertaTokenizer, RobertaModel

proxy = get_secrets("proxy")["proxy"]
os.environ['HTTP_PROXY'] = proxy
os.environ['HTTPS_PROXY'] = proxy

In [ ]:
from transformers import AutoModel, ElectraTokenizer, AutoConfig


model = AutoModel.from_pretrained("sales_seq_electra-small_model")
#lmmodel = AutoModelForMaskedLM.from_pretrained("monologg/koelectra-base-v3-generator")
tokenizer = ElectraTokenizer.from_pretrained("koelectra_tokenizer")

In [ ]:
config = AutoConfig.from_pretrained("sales_seq_electra-small_model", return_dict=True)

In [ ]:
config.hidden_size

In [ ]:
import torch

input_ids = tokenizer("나는 식당에서 밥을 먹었다.").input_ids
md_result = model(torch.tensor([input_ids]))
#lmmd_result = lmmodel(torch.tensor([input_ids]))

In [ ]:
md_result.last_hidden_state.shape

In [ ]:
type(model.config)

In [ ]:
md_result.last_hidden_state

In [ ]:
lmmd_result = lmmodel.electra(torch.tensor([input_ids]))

In [ ]:
lmmd_result.last_hidden_state

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="monologg/koelectra-base-v3-generator",
    tokenizer="monologg/koelectra-base-v3-generator"
)

fill_mask("나는 {} 밥을 먹었다.".format(fill_mask.tokenizer.mask_token))

In [ ]:
from transformers import pipeline
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')
fill_mask = pipeline(
    "fill-mask",
    model='roberta-base'
)

In [ ]:
fill_mask("The man worked as a <mask>.")

In [ ]:
import torch.nn as nn

embedding = nn.Embedding(7, 3)
input = torch.LongTensor([[1, 2, 4, 5], [4, 3, 2, 5]])
a = embedding(input)

In [ ]:
a.shape

In [ ]:
a[0]

In [ ]:
import torch.nn as nn
import torch
import numpy as np

gru = nn.GRU(10, 20, 7, batch_first=True, dropout=0.2)

In [ ]:
a = torch.FloatTensor(np.stack([val['2014-01-09'][1]['input'].T, val['2014-01-09'][2]['input'].T]))
a.shape

In [ ]:
a

In [ ]:
b = torch.randn(7, 2, 20)

In [ ]:
c, d = gru(a, b)

In [ ]:
c.shape

In [ ]:
input_ids = tokenizer.batch_encode_plus(val['2014-01-09'][1]['store_seq'][:2], padding='max_length', max_length=50, truncation=True).input_ids
model(torch.tensor(input_ids)).last_hidden_state[:, 0, :].shape

In [ ]:
aa = torch.stack(list(map(lambda x: model(torch.tensor(input_ids)).last_hidden_state[:, x, :].sum(0), range(7))))
bb = torch.stack(list(map(lambda x: model(torch.tensor(input_ids)).last_hidden_state[:, x, :].sum(0), range(7))))

In [ ]:
cc = torch.stack([aa, bb])
cc.shape

In [ ]:
torch.cat([c, cc], -1).shape

In [ ]:
bert_layer = nn.Linear(config.hidden_size, 20)

In [ ]:
bert_layer(cc)[:, -1]

In [ ]:
from datasets import load_dataset

load_dataset('pickle', 'sales_data/val_7days.pkl')

In [ ]:
import json

json.dumps(val)

In [ ]:
json.dumps({'a': [[1, 2, 3], [4, 5, 6]]})

In [ ]:
list(val['2014-01-09'][1]['input'])